Main Code

In [1]:
import cv2
import numpy as np
import utlis
from scipy.misc import ascent

curveList = []
avgVal = 10

def getLaneCurve(img, display=2):
    imgcopy = img.copy()
    imgResult = img.copy()
    ###step1
    
    imgThres = utlis.thresholding(img)
    ###step2
    hT, wT, c = img.shape
    points = utlis.valTrackbars()
    imgwarp = utlis.warpimg(imgThres, points, wT, hT)
    imgwarppoints = utlis.drawPoints(imgcopy, points)
    ###step 3
    middlepoint, imghist = utlis.getHistogram(imgwarp, display=True, minPer=0.5, region=4)
    curveAveragepoint, imghist = utlis.getHistogram(imgwarp, display=True, minPer=0.9)
    curveRaw = curveAveragepoint - middlepoint
    ###Step 4
    curveList.append(curveRaw)
    if len(curveList) > avgVal:
        curveList.pop(0)
    curve = int(sum(curveList) / len(curveList))
    ###step 5
    if display != 0:
       imgInvWarp = utlis.warpimg(imgwarp, points, wT, hT, inv=True)
       imgInvWarp = cv2.cvtColor(imgInvWarp, cv2.COLOR_GRAY2BGR)
       imgInvWarp[0:hT//3, 0:wT] = 0, 0, 0
       imgLaneColor = np.zeros_like(img)
       imgLaneColor[:] = 0, 255, 0
       imgLaneColor = cv2.bitwise_and(imgInvWarp, imgLaneColor)
       imgResult = cv2.addWeighted(imgResult, 1, imgLaneColor, 1, 0)
       midY = 450
       cv2.putText(imgResult, str(curve), (wT//2-80, 85), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 255), 3)
       cv2.line(imgResult, (wT//2, midY), (wT//2 + (curve * 3), midY), (255, 0, 255), 5)
       cv2.line(imgResult, ((wT // 2 + (curve * 3)), midY - 25), (wT // 2 + (curve * 3), midY + 25), (0, 255, 0), 5)
       for x in range(-30, 30):
           w = wT // 20
           cv2.line(imgResult, (w * x + int(curve // 50), midY - 10),
                    (w * x + int(curve // 50), midY + 10), (0, 0, 255), 2)
       
       #timer = cv2.getTickCount()  # Initialize timer here
       #fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
       #cv2.putText(imgResult, 'FPS ' + str(int(fps)), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (230, 50, 50), 3)
    
    if display == 2:
       imgStacked = utlis.stackImages(0.7, ([img, imgwarppoints, imgwarp],
                                           [imghist, imgLaneColor, imgResult]))
       cv2.imshow('ImageStack', imgStacked)
    elif display == 1:
       cv2.imshow('Result', imgResult)

    ##Normalization
    curve = curve / 100
    if curve > 1: curve = 1
    if curve < -1: curve = -1

    cv2.imshow('img', imgThres)
    cv2.imshow('warp', imgwarp)
    cv2.imshow('warp points', imgwarppoints)
    cv2.imshow('Histogram', imghist)

    return curve

if __name__ == '__main__':
    cap = cv2.VideoCapture(r'xyz.mp4')
    initialTrackBarVals = [171, 67, 48, 169]
    utlis.initializeTrackbars(initialTrackBarVals)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        exit()
    frameCounter = 0

    while True:
        frameCounter += 1
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) == frameCounter:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frameCounter = 0
        success, img = cap.read()
        
        if not success:
            print("Error: Failed to read frame.")
            break

        img = cv2.resize(img, (480, 240))
        curve = getLaneCurve(img, display=1)
        
        cv2.imshow('Vid', img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


hsv color detection:

In [2]:
import cv2
import numpy as np
framewidth=640
frameheight=480
cap=cv2.VideoCapture(r'xyz.mp4')
cap.set(3,framewidth)
cap.set(4,frameheight)
def empty(a):
    pass

cv2.namedWindow("hsv")
cv2.resizeWindow("hsv",640,240)
cv2.createTrackbar("hue min","hsv",0,179,empty)
cv2.createTrackbar("hue max","hsv",179,179,empty)
cv2.createTrackbar("sat min","hsv",0,255,empty)
cv2.createTrackbar("sat max","hsv",255,255,empty)
cv2.createTrackbar("value min","hsv",0,255,empty)
cv2.createTrackbar("value max","hsv",255,255,empty)

frameCounter=0
while True:
    frameCounter+=1
    if cap.get(cv2.CAP_PROP_FRAME_COUNT)==frameCounter:
         cap.set(cv2.CAP_PROP_POS_FRAMES,0)
         frameCounter=0

    _,img=cap.read()
    
    imghsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

    hmin=cv2.getTrackbarPos("hue min","hsv")
    hmax=cv2.getTrackbarPos("hue max","hsv")
    smin=cv2.getTrackbarPos("sat min","hsv")
    smax=cv2.getTrackbarPos("sat max","hsv")
    vmin=cv2.getTrackbarPos("value min","hsv")
    vmax=cv2.getTrackbarPos("value max","hsv")

    lower=np.array([hmin,smin,vmin])
    upper=np.array([hmax,smax,vmax])
    mask=cv2.inRange(imghsv,lower,upper)
    result=cv2.bitwise_and(img,img,mask=mask)
    mask=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
    hstack=np.hstack([result])
    
    #cv2.imshow("result",result)
    #cv2.imshow('hsv color space',imghsv)
    cv2.imshow("horizontal stack",hstack)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()    
cv2.destroyAllWindows()
